# Inter AS EIGRP 

## Topology 
![](pic/inter_as_eigrp.png)

### some baseic facts
1. redistribution is done inside the routing config mode of a running protocol 
2. is redistribution _proto1_ inside _proto2_ config it implies, __injecting routes from proto1 into proto2__
3. for inter-prococol redistribution (__seed__) metric is optional but for inter-ptocol it is mandatory 
4. A selective route redistribution can be done with __Route Map__

## Basic config 

`
! R1
!********************************
conf t 
int s1/0 
ip add 12.0.0.1 255.255.255.252
no sh 
int s1/1 
ip add 13.0.0.1 255.255.255.252
no sh 
exit 
router eigrp 12
no auto 
net 0.0.0.0 
end 
wr`

`
! R2
!********************************
conf t 
int s1/0 
ip add 12.0.0.2 255.255.255.252
no sh 
int l0
ip add 2.2.2.1 255.255.255.0
no sh 
exit 
router eigrp 12
no auto 
net 0.0.0.0 
end 
wr`

`
! R2
!********************************
conf t 
int s1/0 
ip add 13.0.0.2 255.255.255.252
no sh 
int l0
ip add 3.3.3.1 255.255.255.0
no sh 
exit 
router eigrp 13
no auto 
net 0.0.0.0 
end 
wr`

## Routing Tables 

`R1#sh ip route 
     2.0.0.0/24 is subnetted, 1 subnets
D       2.2.2.0 [90/2297856] via 12.0.0.2, 00:19:44, Serial1/0
     3.0.0.0/24 is subnetted, 1 subnets
D       3.3.3.0 [90/2297856] via 13.0.0.2, 00:19:05, Serial1/1
     12.0.0.0/30 is subnetted, 1 subnets
C       12.0.0.0 is directly connected, Serial1/0
     13.0.0.0/30 is subnetted, 1 subnets
C       13.0.0.0 is directly connected, Serial1/1`

`R2#sh ip route 
     2.0.0.0/24 is subnetted, 1 subnets
C       2.2.2.0 is directly connected, Loopback0
     12.0.0.0/30 is subnetted, 1 subnets
C       12.0.0.0 is directly connected, Serial1/0
     13.0.0.0/30 is subnetted, 1 subnets
D       13.0.0.0 [90/2681856] via 12.0.0.1, 00:18:53, Serial1/0`

`R3#sh ip route    
    3.0.0.0/24 is subnetted, 1 subnets
C       3.3.3.0 is directly connected, Loopback0
     12.0.0.0/30 is subnetted, 1 subnets
D       12.0.0.0 [90/2681856] via 13.0.0.1, 00:17:27, Serial1/0
     13.0.0.0/30 is subnetted, 1 subnets
C       13.0.0.0 is directly connected, Serial1/0`

* notice that, R2 cant see 3.3.3.0/24 network and R3 can't see 2.2.2.0/24 network.but R1 can see both. 
* There's no auto redistribution has no auto redistribution 

## Enabling inter-AS redistribution 

`
! configure in R1 (redist point)
! ***************************
router eigrp 12
 redistribute eigrp 13
 exit` <p>
   
`
! configure in R1 (redist point)
! ***************************
 router eigrp 13
 redistribute eigrp 12
 exit`

## check the routing tables 

`R2#sh ip route eigrp
     3.0.0.0/24 is subnetted, 1 subnets
D EX    3.3.3.0 [170/2809856] via 12.0.0.1, 00:16:03, Serial1/0
     13.0.0.0/30 is subnetted, 1 subnets
D       13.0.0.0 [90/2681856] via 12.0.0.1, 00:39:48, Serial1/0`

`R3#sh ip route eigrp
     2.0.0.0/24 is subnetted, 1 subnets
D EX    2.2.2.0 [170/2809856] via 13.0.0.1, 00:16:22, Serial1/0
     12.0.0.0/30 is subnetted, 1 subnets
D       12.0.0.0 [90/2681856] via 13.0.0.1, 00:39:43, Serial1/0
`
* notice __D EX__ routes with metric __170__ is the redistributed routes

# Inter Routing Protocol Redistribution ( RIPv2 - EIGRP) 

* RIP's default metric is Hop count (not so realistic as bandwidth is not considered) 

## topology

![](pic/rip_eigrp.png)

## Basic Config 

### Activating RIPv2 at R4
` s1/0
  int s1/0
  ip add 14.0.0.2 255.255.255.0
  ip add 14.0.0.2 255.255.255.252
  no sh
  int l0
  ip address 4.4.4.1 255.255.255.0
  exit
  router rip
  version 2
  network 0.0.0.0
`

### Activating RIPv2 at R2
` int s1/2
  ip address 14.0.0.1 255.255.255.252
  no sh
  exit
  router rip
  v2
  version 2
  network 0.0.0.0
`

### check the routing tables
`!R1
sh ip route rip
R    4.0.0.0/8 [120/1] via 14.0.0.2, 00:00:29, Serial1/2
`
<p>
    
`!R4
R4#sh ip route rip
R    12.0.0.0/8 [120/1] via 14.0.0.1, 00:00:12, Serial1/0
R    13.0.0.0/8 [120/1] via 14.0.0.1, 00:00:12, Serial1/0`


## Perform Inter-protocol reditribution 

* while redistributing a protocol into another, careful seed matrix is important to perform optimal redistribution
* a flaued seed metric may destry routing descissions 
* EIGRP metrics are : Bandwidth, Delay, Load, Reliability & MTU
* out of 5, __BW, DLY & MTU__ is static get it from corresponding interface 
* keep Load = 1 (min) and Reliability = 255 (max) while config 

### step 1 : Gather seed metric values of EIGRP for RIP route
`
R1#sh int s1/0 | inc MTU
  MTU 1500 bytes, BW 1544 Kbit/sec, DLY 20000 usec
`

### step 2 : redistribute RIP into all EIGRP ASs
BW measured in kbps, DLY measured in 10 usec

`
router eigrp 13
  redistribute rip metric 1544 2000 255 1 1500
router eigrp 12
  redistribute rip metric 1544 2000 255 1 1500
`

### step 3 : redistribute EIGRP into RIP domain 
metric is hop count, since RIP router is 1 hop away from redistribution point, thus metric = 1

`router rip
  version 2
  redistribute eigrp 12 metric 1
  redistribute eigrp 13 metric 1
  router eigrp 12`
  
### step 4 : verify routing tables 
__From R2__

`     2.0.0.0/24 is subnetted, 1 subnets
C       2.2.2.0 is directly connected, Loopback0
     3.0.0.0/24 is subnetted, 1 subnets
D EX    3.3.3.0 [170/2809856] via 12.0.0.1, 01:45:42, Serial1/0
     4.0.0.0/24 is subnetted, 1 subnets
D EX    4.4.4.0 [170/2681856] via 12.0.0.1, 00:00:12, Serial1/0
     12.0.0.0/30 is subnetted, 1 subnets
C       12.0.0.0 is directly connected, Serial1/0
     13.0.0.0/30 is subnetted, 1 subnets
D       13.0.0.0 [90/2681856] via 12.0.0.1, 02:09:26, Serial1/0
     14.0.0.0/30 is subnetted, 1 subnets
D       14.0.0.0 [90/2681856] via 12.0.0.1, 00:26:16, Serial1/0
`

__from R4__

`     2.0.0.0/24 is subnetted, 1 subnets
R       2.2.2.0 [120/1] via 14.0.0.1, 00:00:09, Serial1/0
     3.0.0.0/24 is subnetted, 1 subnets
R       3.3.3.0 [120/1] via 14.0.0.1, 00:00:05, Serial1/0
     4.0.0.0/24 is subnetted, 1 subnets
C       4.4.4.0 is directly connected, Loopback0
     12.0.0.0/30 is subnetted, 1 subnets
R       12.0.0.0 [120/1] via 14.0.0.1, 00:00:17, Serial1/0
     13.0.0.0/30 is subnetted, 1 subnets
R       13.0.0.0 [120/1] via 14.0.0.1, 00:00:17, Serial1/0
     14.0.0.0/30 is subnetted, 1 subnets
C       14.0.0.0 is directly connected, Serial1/0`

### step 5  : test round trip reachability
* seeing routes in table is not enough, packet must reach and come back. thus do a __Ping Test__ 
* here pinging a network in RIP domain from EIGRP domain
* a common fault of not having a ping back if the redistribution is not set both way
* a __`debug ip packet`__ command is useful, you must an entry with subjected source, destination  packet with status __Unable__

`R2#ping 4.4.4.1
Type escape sequence to abort.
Sending 5, 100-byte ICMP Echos to 4.4.4.1, timeout is 2 seconds:
!!!!!
Success rate is 100 percent (5/5), round-trip min/avg/max = 88/92/104 ms
`

# Reditribute Inter Routing Protocol Redistribution ( RIPv2 - EIGRP - OSPF) 

## Topology
![](pic/rip-eigrp-ospf.png)

### Configure OSPF in R5 & R1
__R1__

`int s1/3
  ip add 15.0.0.1 255.255.255.252
  no sh
exit
  router ospf 1
  network 0.0.0.0 0.0.0.0 area 0`



__R5__

`
int s1/0
  ip address 15.0.0.2 255.255.255.252
  no sh
  int l0
  ip add 5.5.5.1
  ip add 5.5.5.1 255.255.255.0
exit
  router ospf 1
  network 0.0.0.0 0.0.0.0 area 0
`


### Redistribute OSPF into EIGRP & RIP 

`
router eigrp 12
  redistribute ospf 1 metric 1544 2000 255 1  1500
  exit
router eigrp 13
  redistribute ospf 1 metric 1544 2000 255 1  1500
  exit
router rip
  ver 2
  redistribute ospf 1 metric 1`
  
### Redistribute EIGRP & RIP into OSPF
The __subnet__ command is important to make sure redistributing __classless routes__ 

`  
router ospf 1
  redistribute rip metric 1000 subnets
  redistribute eigrp 13 metric 1000 subnets
  redistribute eigrp 12 metric 1000 subnets`

### verify Routing tables 

__From R2__

`R2#sh ip route eigrp
     3.0.0.0/24 is subnetted, 1 subnets
D EX    3.3.3.0 [170/2809856] via 12.0.0.1, 02:46:24, Serial1/0
     4.0.0.0/24 is subnetted, 1 subnets
D EX    4.4.4.0 [170/2681856] via 12.0.0.1, 01:00:55, Serial1/0
     5.0.0.0/32 is subnetted, 1 subnets
D EX    5.5.5.1 [170/2681856] via 12.0.0.1, 00:10:09, Serial1/0
     13.0.0.0/30 is subnetted, 1 subnets
D       13.0.0.0 [90/2681856] via 12.0.0.1, 03:10:09, Serial1/0
     14.0.0.0/30 is subnetted, 1 subnets
D       14.0.0.0 [90/2681856] via 12.0.0.1, 01:26:57, Serial1/0
     15.0.0.0/30 is subnetted, 1 subnets
D       15.0.0.0 [90/2681856] via 12.0.0.1, 00:20:56, Serial1/0`

__From R4__

`R4#sh ip route rip
     2.0.0.0/24 is subnetted, 1 subnets
R       2.2.2.0 [120/1] via 14.0.0.1, 00:00:06, Serial1/0
     3.0.0.0/24 is subnetted, 1 subnets
R       3.3.3.0 [120/1] via 14.0.0.1, 00:00:06, Serial1/0
     5.0.0.0/32 is subnetted, 1 subnets
R       5.5.5.1 [120/1] via 14.0.0.1, 00:00:06, Serial1/0
     12.0.0.0/30 is subnetted, 1 subnets
R       12.0.0.0 [120/1] via 14.0.0.1, 00:00:06, Serial1/0
     13.0.0.0/30 is subnetted, 1 subnets
R       13.0.0.0 [120/1] via 14.0.0.1, 00:00:07, Serial1/0
     15.0.0.0/30 is subnetted, 1 subnets
R       15.0.0.0 [120/1] via 14.0.0.1, 00:00:07, Serial1/0`

__From R5__
1. check the metric at routing table is 1000 as given in Command
2. E2 is OSPF external routes (type 2) - distance from ASBR, E1 : Distance from local router 

`R5#sh ip route ospf
     2.0.0.0/24 is subnetted, 1 subnets
O E2    2.2.2.0 [110/1000] via 15.0.0.1, 00:03:52, Serial1/0
     3.0.0.0/24 is subnetted, 1 subnets
O E2    3.3.3.0 [110/1000] via 15.0.0.1, 00:04:02, Serial1/0
     4.0.0.0/24 is subnetted, 1 subnets
O E2    4.4.4.0 [110/1000] via 15.0.0.1, 00:04:38, Serial1/0
     12.0.0.0/30 is subnetted, 1 subnets
O       12.0.0.0 [110/128] via 15.0.0.1, 00:08:58, Serial1/0
     13.0.0.0/30 is subnetted, 1 subnets
O       13.0.0.0 [110/128] via 15.0.0.1, 00:08:58, Serial1/0
     14.0.0.0/30 is subnetted, 1 subnets
O       14.0.0.0 [110/128] via 15.0.0.1, 00:08:58, Serial1/0`

### verify round trip reachability 

`R2#traceroute 5.5.5.1
Type escape sequence to abort.
Tracing the route to 5.5.5.1
  1 12.0.0.1 4 msec 16 msec 20 msec
  2 15.0.0.2 32 msec 36 msec 36 msec
`

# Sub-optimal Routing & Routing Loop

* sometimes redistribution may create routing loop, i.e. dorwaring packet back and forth between routers 
* sub-optimal routing is thus used, forcing packet to take a non-Best path to reach the destination 
* ping can't detact routing loop
* Traceroute can detect, you'll see same address appearing again an again 

## Topology
* Interfaces are configured 
* IGP is setup 
* bidirectional reditribution is setup 
![](pic\SOR.png)